In [ ]:
from fastai.vision.all import *
import pandas as pd
%matplotlib inline
set_seed(3865)

In [ ]:
train_path='../input/resized-2015-2019-diabetic-retinopathy-detection/resized_traintest15_train19'
train_df = pd.read_csv('../input/resized-2015-2019-diabetic-retinopathy-detection/labels/traintestLabels15_trainLabels19.csv')
# for interactive DEBUG: reduce amount of train images : train_df = train_df[:1024]

In [ ]:
train_df

In [ ]:
import albumentations as Alb
class AlbTransform(Transform):
    def __init__(self, aug): self.aug = aug
    def encodes(self, img: PILImage):
        aug_img = self.aug(image=np.array(img))['image']
        return PILImage.create(aug_img)
    
def get_augs(): return Alb.Compose([
    Alb.ShiftScaleRotate(rotate_limit=20, border_mode=0, value=(0,0,0) ),
    # Alb.Transpose(),
    Alb.HorizontalFlip(),
    # Alb.RandomRotate90(),
    Alb.RandomBrightnessContrast(),
    Alb.HueSaturationValue(
      hue_shift_limit=5, 
      sat_shift_limit=5, 
      val_shift_limit=5 ),
])

item_tfms = [Resize(224), AlbTransform(get_augs())] 
batch_tfms = Normalize.from_stats(*imagenet_stats) 

In [ ]:
dls = ImageDataLoaders.from_df(train_df, path=train_path, 
    fn_col='image', label_col='level', suff='.jpg', valid_pct=0.1, bs=32,
    item_tfms=item_tfms, batch_tfms=batch_tfms, shuffle=True)

In [ ]:
# uncomment to test data loaders
dls.train.show_batch(max_n=12)
print('train items:', len(dls.train.items), 'validation items:', len(dls.valid.items))
# dls.valid.show_batch(max_n=12)

In [ ]:
dls.vocab
# dls.c

In [ ]:
learn = vision_learner(dls, resnet34, path='.', 
    loss_func=FocalLoss(),  
    metrics=[accuracy]  )

In [ ]:
# lrs = learn.lr_find(suggest_funcs=(minimum, steep, valley, slide))
# print('lrs.valley=', lrs.valley)

In [ ]:
learn.fine_tune(4, cbs=[ShowGraphCallback()])

In [ ]:
learn.save('retinopathy201519-rn34-1') 